## Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Python Libraries

In [2]:
import numpy as np
import pandas as pd
import os
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
import missingno
import warnings
warnings.filterwarnings('ignore')

## Preprocessing data

#### Load Data

In [17]:
data = pd.read_csv("/content/drive/MyDrive/Classroom/IS252/Đồ Án Data Mining/Click_Through_Rate Prediction Data/ad_10000records.csv")
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,2016-06-09 21:43:05,0
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,2016-01-16 17:56:05,0
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,2016-06-29 10:50:45,0
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,2016-06-21 14:32:32,0
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,2016-07-21 10:54:35,1


In [18]:
print("Số cột của dữ liệu: ", data.shape[1])
print("Số dòng của dữ liệu: ", data.shape[0])

Số cột của dữ liệu:  10
Số dòng của dữ liệu:  10000


#### Check data_type and null_value


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  10000 non-null  float64
 1   Age                       10000 non-null  float64
 2   Area Income               10000 non-null  float64
 3   Daily Internet Usage      10000 non-null  float64
 4   Ad Topic Line             10000 non-null  object 
 5   City                      10000 non-null  object 
 6   Gender                    10000 non-null  object 
 7   Country                   10000 non-null  object 
 8   Timestamp                 10000 non-null  object 
 9   Clicked on Ad             10000 non-null  int64  
dtypes: float64(4), int64(1), object(5)
memory usage: 781.4+ KB


In [20]:
# Timestamp bị sai kiểu dữ liệu
data['Timestamp'] = pd.to_datetime(data['Timestamp'], errors='coerce')

#### Check outliers

In [21]:
def detect_outlier(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)

    # Tính IQR
    IQR = Q3 - Q1

    # Tính ngưỡng dưới và ngưỡng trên
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    # Xác định phần tử ngoại lai
    outliers = [x for x in data if x < lower_bound or x > upper_bound]
    outliers = outliers
    return outliers

In [22]:
number_variable = data.select_dtypes(include = 'number')
category_variable = data.select_dtypes(include = 'object')
number_variable

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad
0,62.26,32.0,69481.85,172.83,0
1,41.73,31.0,61840.26,207.17,0
2,44.40,30.0,57877.15,172.83,0
3,59.88,28.0,56180.93,207.17,0
4,49.21,30.0,54324.73,201.58,1
...,...,...,...,...,...
9995,41.73,31.0,61840.26,207.17,1
9996,41.73,28.0,51501.38,120.49,0
9997,55.60,39.0,38067.08,124.44,0
9998,46.61,50.0,43974.49,123.13,1


In [23]:
for col in number_variable.columns:
    print(f"{col}: ", detect_outlier(number_variable[col]))

Daily Time Spent on Site:  []
Age:  []
Area Income:  [14548.06, 14548.06, 14548.06, 14548.06, 14548.06, 14548.06, 14775.5, 13996.5, 14548.06, 14548.06, 14775.5, 14548.06, 14548.06, 14548.06, 14775.5, 14775.5, 14548.06, 14548.06, 14548.06, 14548.06, 14775.5, 14548.06, 14548.06, 14775.5, 14775.5, 14548.06, 14548.06, 14775.5, 14775.5, 13996.5, 14548.06, 13996.5, 14548.06, 14548.06]
Daily Internet Usage:  []
Clicked on Ad:  []


##### => Mặc dù thuộc tính Area Income có rất nhiều phần tử ngoại lai và thay bằng giá trị trung vị dựa trên độ lệch

In [24]:
income_outliers = detect_outlier(number_variable['Area Income'])

In [25]:
median_value = data['Area Income'].median()
data['Area Income'] = data['Area Income'].apply(lambda x: median_value if x in income_outliers else x)

#### Check consistency


In [26]:
columns_with_values_lower_than_zero = number_variable.columns[number_variable.lt(0).any()]
columns_with_values_lower_than_zero

Index([], dtype='object')

#### Check valid_data

In [27]:
for col_name in category_variable.columns:
    print("----------------")
    print(category_variable[col_name].value_counts())

----------------
Ad Topic Line
Cloned explicit middleware                 344
Streamlined homogeneous analyzer           212
Business-focused transitional solution     207
Sharable reciprocal project                187
Intuitive exuding service-desk             173
                                          ... 
Ergonomic neutral portal                     1
Implemented uniform synergy                  1
Visionary multi-tasking alliance             1
Inverse zero tolerance customer loyalty      1
Reactive interactive protocol                1
Name: count, Length: 559, dtype: int64
----------------
City
Hubbardmouth       336
West Brandonton    275
Lisafort           261
Wintersfort        230
Stewartbury        207
                  ... 
Turnerview           1
North Tiffany        1
North Kristine       1
East Troyhaven       1
Davidview            1
Name: count, Length: 521, dtype: int64
----------------
Gender
Female    5376
Male      4624
Name: count, dtype: int64
----------------
Co

In [28]:
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,2016-06-09 21:43:05,0
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,2016-01-16 17:56:05,0
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,2016-06-29 10:50:45,0
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,2016-06-21 14:32:32,0
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,2016-07-21 10:54:35,1


#### Convert data_type

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Classroom/IS252/Đồ Án Data Mining/Click_Through_Rate Prediction Data/Cleaned_Data.csv")
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Timestamp,Clicked on Ad,date,time
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,2016-06-09 21:43:05,0,2016-06-09,21:43:05
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,2016-01-16 17:56:05,0,2016-01-16,17:56:05
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,2016-06-29 10:50:45,0,2016-06-29,10:50:45
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,2016-06-21 14:32:32,0,2016-06-21,14:32:32
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,2016-07-21 10:54:35,1,2016-07-21,10:54:35


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Daily Time Spent on Site  10000 non-null  float64       
 1   Age                       10000 non-null  float64       
 2   Area Income               10000 non-null  float64       
 3   Daily Internet Usage      10000 non-null  float64       
 4   Ad Topic Line             10000 non-null  object        
 5   City                      10000 non-null  object        
 6   Gender                    10000 non-null  object        
 7   Country                   10000 non-null  object        
 8   Timestamp                 10000 non-null  datetime64[ns]
 9   Clicked on Ad             10000 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 781.4+ KB


In [30]:
categorical_features = ['Ad Topic Line',
                        'City',
                        'Gender',
                        'Country']
numerical_features = ['Daily_Time_Spent_on_Site',
                      'Age',
                      'Area_Income',
                      'Daily_Internet_Usage']
time_feature = ['Timestamp']

In [31]:
for df_obj in categorical_features:
    data[df_obj] = data[df_obj].astype("category")

In [32]:
for df_time in time_feature:
    data[df_time] = pd.to_datetime(data[df_time])

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Daily Time Spent on Site  10000 non-null  float64       
 1   Age                       10000 non-null  float64       
 2   Area Income               10000 non-null  float64       
 3   Daily Internet Usage      10000 non-null  float64       
 4   Ad Topic Line             10000 non-null  category      
 5   City                      10000 non-null  category      
 6   Gender                    10000 non-null  category      
 7   Country                   10000 non-null  category      
 8   Timestamp                 10000 non-null  datetime64[ns]
 9   Clicked on Ad             10000 non-null  int64         
dtypes: category(4), datetime64[ns](1), float64(4), int64(1)
memory usage: 587.8 KB


#### Extract data[Timestamp] to "year", "month", "day", "hour"


In [34]:
data['year'] = data['Timestamp'].dt.year
data['month'] = data['Timestamp'].dt.month
data['day'] = data['Timestamp'].dt.day
data['hour'] = data['Timestamp'].dt.hour

#### Drop columns doesn't affect the results


In [35]:
data['year'].value_counts()

,count
year,
2016,10000


In [36]:
data_to_drops = ['Timestamp', 'year']
data = data.drop(columns=data_to_drops)

In [37]:
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Clicked on Ad,month,day,hour
0,62.26,32.0,69481.85,172.83,Decentralized real-time circuit,Lisafort,Male,Svalbard & Jan Mayen Islands,0,6,9,21
1,41.73,31.0,61840.26,207.17,Optional full-range projection,West Angelabury,Male,Singapore,0,1,16,17
2,44.40,30.0,57877.15,172.83,Total 5thgeneration standardization,Reyesfurt,Female,Guadeloupe,0,6,29,10
3,59.88,28.0,56180.93,207.17,Balanced empowering success,New Michael,Female,Zambia,0,6,21,14
4,49.21,30.0,54324.73,201.58,Total 5thgeneration standardization,West Richard,Female,Qatar,1,7,21,10


#### Convert to Category Codes

In [38]:
for cat_col in ["Ad Topic Line", "City", "Gender", "Country"]:
    print(data[cat_col].value_counts())
    data[cat_col] = data[cat_col].cat.codes
    data[cat_col] = data[cat_col].astype("category")

Ad Topic Line
Cloned explicit middleware                344
Streamlined homogeneous analyzer          212
Business-focused transitional solution    207
Sharable reciprocal project               187
Intuitive exuding service-desk            173
                                         ... 
Profound stable product                     1
Progressive 24hour forecast                 1
Progressive empowering alliance             1
Public-key zero-defect analyzer             1
Visionary multi-tasking alliance            1
Name: count, Length: 559, dtype: int64
City
Hubbardmouth         336
West Brandonton      275
Lisafort             261
Wintersfort          230
Stewartbury          207
                    ... 
East Troyhaven         1
South Ronald           1
South Pamela           1
Josephberg             1
South Christopher      1
Name: count, Length: 521, dtype: int64
Gender
Female    5376
Male      4624
Name: count, dtype: int64
Country
Australia         388
Czech Republic    330
Turkey 

#### Encoding cat variables

In [39]:
label_encoders = {}
for column in ['Ad Topic Line', 'City', 'Gender', 'Country', 'month', 'day', 'hour']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [ ]:
data.to_csv("/content/drive/MyDrive/Classroom/IS252/Đồ Án Data Mining/Click_Through_Rate Prediction Data/Clean_Data_V2.csv", index = False)